### Import the necessary libraries

In [ ]:
!pip install -r requirements.txt

  Using cached huggingface_hub-0.23.1-py3-none-any.whl (401 kB)
  Using cached tokenizers-0.19.1-cp310-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.0
    Uninstalling huggingface-hub-0.20.0:
      Successfully uninstalled huggingface-hub-0.20.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastembed 0.2.7 requires huggingface-hub<0.21,>=0.20, but you have huggingface-hub 0.23.1 which is incompatible.
fastembed 0.2.7 requires tokenizers<0.16,>=0.15, but you have tokenizers 0.19.1 which is incompatible.
You should consider upgrading via the 'C:\Users\noora\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
!pip install sentence-transformers

You should consider upgrading via the 'C:\Users\noora\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

2024-05-26 21:59:26 - Loaded .env file


In [ ]:
load_dotenv()

True

In [ ]:
groq_api_key = os.environ['GROQ_API_KEY']

In [ ]:
llm_groq = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

In [ ]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("pdf")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[10]

Document(page_content='VI. BUsINEss ENVIRoNMENt:\nthe year that was:\nIt has been an eventful year for both global and domestic economy which was marked by events like Brexit, referendums in \nEurozone, geo-political issues and Presidential Elections in United States, UK & France and Demonetization in India. Despite these uncertainties, we saw Indian economy emerge as one of the best performing economy amongst the emerging markets.', metadata={'source': 'pdf\\amc-annual-report---2016---17.pdf', 'page': 4})

In [ ]:
#embeddings = OpenAIEmbeddings()
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

c:\Users\noora\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-05-26 21:59:55 - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


c:\Users\noora\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
#embedding = FastEmbedEmbeddings()

In [ ]:
vectorstore=FAISS.from_documents(final_documents ,huggingface_embeddings)
vectorstore

Batches: 100%|██████████| 90/90 [09:40<00:00,  6.45s/it]

2024-05-26 22:09:40 - Loading faiss with AVX2 support.
2024-05-26 22:09:40 - Successfully loaded faiss with AVX2 support.


In [ ]:
prompt = ChatPromptTemplate.from_template(
"""
You are Question-answering bot.
I want you to ptovide accurate and precise response from the document for the query given by the user.
<context>
{context}
<context>
Questions:{input}

"""
)

In [ ]:
document_chain=create_stuff_documents_chain(llm_groq, prompt)

In [ ]:
retriever=vectorstore.as_retriever()

In [ ]:
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
response=retrieval_chain.invoke({'input':"Print the table CONSOLIDATED BALANCE SHEET AS AT 31 MARCH 2019?"})
print(response['answer'])

Batches: 100%|██████████| 1/1 [00:00<00:00, 13.88it/s]


2024-05-26 22:23:40 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Here is the table:

**CONSOLIDATED BALANCE SHEET AS AT 31 MARCH 2019**

(` In Lakh)

**Particulars** | **Note** | **As at 31 Mar 2019** | **As at 31 Mar 2018** | **As at 01 April 2017**
-------------------|----------|---------------------|---------------------|---------------------
**ASSETS**
(1) Financial Assets |  |  |  | 
(a) Cash & Cash equivalents | 2 | 607.33 | 405.82 | 765.34 
(b) Bank Balance other than (a) above | 3 | 10,324.26 | 7,619.21 | 8,574.60 
(c) Receivables | 4 | 2,759.16 | 10,030.32 | 2,363.11 
(d) Investments | 5 | 1,09,349.01 | 66,079.72 | 53,953.97 
(e) Other Financial assets | 6 | 730.13 | 705.60 | 577.55 
(2) Non Financial Assets |  |  |  | 
(a) Current Tax assets (Net) | 7 | 3,633.84 | 1,910.40 | 1,851.45 
(b) Deferred Tax assets (Net) | 8 |  | 183.01 |  |
(c) Property, Plant and Equipment | 9 | 17,260.77 | 16,623.51 | 16,618.11 
(d) Capital work-in-progre

In [ ]:
response=retrieval_chain.invoke({'input':"Print the table RIGHT TO USE ASSET - AS AT 31 MAR 2022?"})
print(response['answer'])

Batches: 100%|██████████| 1/1 [00:00<00:00, 24.38it/s]


2024-05-26 22:25:34 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Here is the table RIGHT TO USE ASSET - AS AT 31 MAR 2022:

**RIGHT TO USE ASSET - AS AT 31 MAR 2022**

**Particulars** | **Gross Block (At Cost)** | **Accumulated Depreciation** | **Net Block**
---------------------|-------------------------|-------------------------|---------------------
**Building** | 10,329.95 | 3,082.59 | 7,247.36
**Computers** | 724.33 | 52.20 | 672.13
**Vehicle** | 107.44 | - | 107.44
**Grand Total** | 11,161.72 | 3,134.79 | 7,026.93


In [ ]:
response=retrieval_chain.invoke({'input':"Print the table for MATURITY ANALYSIS  OF ASSETS AND LIABILITIES As at March 31, 2020?"})
print(response['answer'])

Batches: 100%|██████████| 1/1 [00:00<00:00,  8.34it/s]


2024-05-26 22:27:15 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Based on the provided context, I will print the table for MATURITY ANALYSIS  OF ASSETS AND LIABILITIES As at March 31, 2020:

**MATURITY ANALYSIS  OF ASSETS AND LIABILITIES**
**As at March 31, 2020**

**ASSETS**

**(1) Financial Assets**

| Particulars | Within 12 months | After 12 months | Total |
| --- | --- | --- | --- |
| Cash & Cash equivalents | - | 783.37 | 783.37 |
| Bank Balance other than above | 2,174.43 | - | 2,174.43 |
| Receivables | 4,032.39 | - | 4,032.39 |
| Other Receivables | - | - | - |
| Investments | 11,217.85 | 163,710.03 | 174,927.88 |

**Total Financial Assets** | - | 174,927.88 | 174,927.88 |

**Liabilities**

| (Information not provided in the given context |  |

Please note that the table only presents the information available in the provided context. If you would like to ask another question or seek further clarification, please feel free to do so.


In [ ]:
response=retrieval_chain.invoke({'input':"Print the table for RESERVES AND SURPLUS for the year 2017-18?"})
print(response['answer'])

Batches: 100%|██████████| 1/1 [00:00<00:00, 20.49it/s]


2024-05-26 22:28:57 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Based on the provided context, the table for RESERVES AND SURPLUS for the year 2017-18 is:

SBI FUNDS MANAGEMENT PRIVATE LIMITED44
3 RESERVES AND SURPLUS  
 As at 
31-Mar-2018 As at 
31-Mar-2017
` (in lacs) ` (in lacs)
General Reserve
Balance at the beginning of the year  9,260.09  8,760.09 
Add: Transfer from statement of profit and loss  500.00  500.00 
Balance at the end of the year (A)  9,760.09  9,260.09 
Surplus in statement of profit and loss
Balance at the beginning of the year  62,837.75  49,330.95 
Add: Profit  for the year  33,103.31  22,431.86 
Less: Transfer to general reserve  500.00  500.00 
Less: Interim dividend  7,000.00  7,000.00 
Less: Tax on interim dividend  1,425.06  1,425.06 
Balance at the end of the year (B)  87,016.00  62,837.75 
Share option outstanding account (Employee stock options)
Balance at the beginning of the year  -    -   
Additions during the

In [ ]:
response=retrieval_chain.invoke({'input':"Print the table for 8. DEFERRED TAX for the year 2020-21?"})
print(response['answer'])

Batches: 100%|██████████| 1/1 [00:00<00:00, 22.95it/s]


2024-05-26 22:31:18 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Here is the table for 8. DEFERRED TAX for the year 2020-21:

**Deferred Tax**

**Particulars**         **As at 31 Mar 2021**         **As at 31 Mar 2020**

* Deferred Tax Assets/(Liabilities) (net)         **(1,801.63)**         **(334.07)**

Here is the breakdown of the deferred tax assets and liabilities:

* **Provision for compensated absences**         **613.24**         **537.77**
* **Provision for gratuity**         **75.67**         **42.08**
* **Amortization /Depreciation on tangible /intangible assets**         **56.30**         **(45.15)**
* **Others including 40(a) and 43B items**         **702.40**         **451.42**
* **Fair value of Investment (Net)**         **(3,378.19)**         **(1,395.84)**
* **Security Deposit**         **2.46**         **2.20**
* **Right to use assets**         **126.49**         **73.45**

Let me know if you have any further questions!


In [ ]:
response=retrieval_chain.invoke({'input':"Print the table for 14. OTHER FINANCIAL LIABILITIES for the year 2022-23?"})
print(response['answer'])

Batches: 100%|██████████| 1/1 [00:00<00:00, 15.54it/s]


2024-05-26 22:32:23 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Based on the provided context, I can extract the relevant table for you. Here is the table for 14. OTHER FINANCIAL LIABILITIES for the year 2022-23:

**14. OTHER FINANCIAL LIABILITIES**

**Particulars**                             **As at 31 Mar 2023**  **As at 31 Mar 2022**

* CSR Payable                         $ 365.75          $ 458.95
* Other liabilities
    i. Earnest money deposit      * $ 2.45          $ 1.97
    ii. Retention money *         $ 35.03          $ 23.23
    iii. Interim Dividend payable * $ 2.30          $ -
    iv. Creditors for capital expenditure* $ 37.02          $ 150.10
    v. Lease liabilities        $ 8,392.67          $ 7,342.66
    vi. Expenses payable          $ 38.53          $ 93.24
* Total                          $ 8,873.75          $ 8,070.15


In [ ]:
response=retrieval_chain.invoke({'input':"Print the table for CONSOLIDATED CASH FLOW STATEMENT FOR THE YEAR ENDED MARCH 31, 2017?"})
print(response['answer'])

Batches: 100%|██████████| 1/1 [00:00<00:00, 24.25it/s]


2024-05-26 22:35:34 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Based on the provided document, here is the table for the CONSOLIDATED CASH FLOW STATEMENT FOR THE YEAR ENDED MARCH 31, 2017:

**CONSOLIDATED CASH FLOW STATEMENT FOR THE YEAR ENDED MARCH 31, 2017**

**Particulars** | **For Year end 31-Mar-2017** | **For Year end 31-Mar-2016**

**Cash Flow from Operating Activities**

* Net profit Before Tax | 32,967.48 | 25,098.22
* Adjustments for:
	+ Depreciation and amortisation | 1,260.74 | 974.71
	+ (Profit) /Loss on sale of fixed assets (Net) | (1.54) | (0.44)
	+ Gains on sale /redemption of current investments | (3,050.49) | (361.95)
	+ Gains on sale /redemption of non-current investments | (143.34) | -
	+ Interest income | (319.25) | (127.18)
	+ Dividend income from current investments | (468.95) | -
	+ Dividend income from non-current investments | (90.55) | (418.39)
* Operating profit before working capital changes | 30,154.10 | 25,164.9

In [ ]:
response=retrieval_chain.invoke({'input':"Print the table for 9.1  PROPERTY, PLANT AND EQUIPMENT - AS AT 31 MAR 2021?"})
print(response['answer'])

Batches: 100%|██████████| 1/1 [00:00<00:00, 24.82it/s]


2024-05-26 22:36:57 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Here is the table for 9.1  PROPERTY, PLANT AND EQUIPMENT - AS AT 31 MAR 2021:

```
Particulars          Gross Block (At Cost)  Accumulated Depreciation  Net Block
As at 01.04.2020    15,018.24          -              -        15,018.24
Additions for the year    -              -              -        -
Deductions/Adjustments/Reclassification for the year    -              -        -
As at 31.03.2021    15,018.24          801.15         267.05     1,068.20
Upto 01.04.2020    15,018.24          534.10         267.05     14,217.09
For the year    -              -              -        -
Deductions/Adjustments/Reclassification for the year    -              -        -
Upto 31.03.2021    15,018.24          801.15         267.05     1,068.20
As at 31.03.2021    15,018.24          801.15         267.05     1,068.20

Building (refer note i )    15,018.24          -              -        15

In [ ]:
response=retrieval_chain.invoke({'input':"Print the table for 9.3 DEPRECIATION AND AMORTIZATION EXPENSE for 2020-21?"})
print(response['answer'])

Batches: 100%|██████████| 1/1 [00:00<00:00, 20.78it/s]


2024-05-26 22:38:06 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Here is the table for 9.3 DEPRECIATION AND AMORTIZATION EXPENSE for 2020-21:

**9.3 DEPRECIATION AND AMORTIZATION EXPENSE**

* For the year ended 31 Mar 2021
* For the year ended 31 Mar 2020

**Depreciation on Property, Plant and Equipment** | **2021** | **2020**
-------------------------|----------|----------
Depreciation | 1,324.05 | 1,327.60

**Amortization of Intangible Assets** | **2021** | **2020**
-------------------------|----------|----------
Amortization | 307.91 | 352.58

**Right to use Assets** | **2021** | **2020**
-------------------------|----------|----------
Right to use Assets | 1,537.17 | 1,344.45

**Total** | **2021** | **2020**
-------------------------|----------|----------
Total | 3,169.13 | 3,024.63
